#### Dependencies

In [48]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import time
import multiprocessing as mp
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import sys
from contextlib import closing
import psutil
import tracemalloc
import threading
import gc

from essentia.standard import (
    MonoLoader,
    Danceability,
    Spectrum,
    FrameCutter,
    Loudness,
    RhythmExtractor2013,
    KeyExtractor,
    Energy,
    TonalExtractor,
    Inharmonicity,
    MFCC,
    OnsetRate,
    SpectralCentroidTime,
    DynamicComplexity,
    SpectralPeaks,
    NoveltyCurve,
    Spectrum,
    FrameGenerator,
    Windowing,
    MelBands,
    BeatsLoudness,
    Beatogram,
    Meter,
)

#### Global Constants

In [49]:
load_dotenv()
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')
CPU_THREADS = int(os.getenv('CPU_THREADS'))

#### Data

In [50]:
songs_data = pd.read_csv('data/songs_final.csv')

#### Feature Extraction Functions

In [51]:
def create_spectrogram_image(spectrogram_db, sample_rate):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', fmax=11025)
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Mel-Spectrogram")
    plt.tight_layout()
    plt.show()
    plt.close()

In [52]:
def mp3_to_spectrogram(audio_path, sample_rate, create_image=False):
    mp3, _ = librosa.load(audio_path, sr=sample_rate)
    spectrogram = librosa.feature.melspectrogram(y=mp3, sr=sample_rate, n_mels=128, fmax=11025)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    if create_image:
        create_spectrogram_image(spectrogram_db, sample_rate)

    return spectrogram_db

In [53]:
def get_mel_bands(audio):
    spectrum = Spectrum()
    frame_generator = FrameGenerator(audio, frameSize=2048, hopSize=1024)
    window = Windowing(type='hann')

    mel_bands = MelBands(numberBands=40)
    mel_band_energies = []

    for frame in frame_generator:
        spec = spectrum(window(frame))
        mel_band_energies.append(mel_bands(spec))

    mel_band_energies = np.array(mel_band_energies)

    return mel_band_energies

In [54]:
def run_essentia_algorithms(audio44k, audio16k):
    _, mfcc_coeffs = MFCC(inputSize=len(audio16k))(audio16k)
    danceability_score = Danceability()(audio44k)
    loudness_score = Loudness()(audio16k)
    bpm, beat_positions, _, _, _ = RhythmExtractor2013(method="multifeature")(audio44k)
    key, scale, _ = KeyExtractor()(audio44k)
    energy_score = Energy()(audio16k)

    ### Chord Significances
    _, _, _, _, chords, _, _, _, _, _, _, _ = TonalExtractor()(audio44k)
    unique_chords, counts = np.unique(chords, return_counts=True)
    chords_significance = {chord: significance for (chord, significance) in zip(unique_chords, counts)}

    ### Inharmonicity
    frames = []
    frameCutter = FrameCutter()
    while True:
        frame = frameCutter(audio44k)
        if not len(frame):
            break
        frames.append(frame)
        
    spectrum_magnitudes = []
    for frame in frames:
        spectrum_magnitudes_frame = Spectrum()(frame)
        spectrum_magnitudes.append(spectrum_magnitudes_frame)
    spectrum_magnitudes = np.array(spectrum_magnitudes).flatten()
    
    frequencies, magnitudes = SpectralPeaks()(audio44k)
    hnr_score = None
    if len(frequencies) > 0 and frequencies[0]: 
        hnr_score = Inharmonicity()(frequencies, magnitudes)
    ###
    
    onset_rate_score = OnsetRate()(audio44k)
    brightness_score = SpectralCentroidTime()(audio44k)
    dynamic_complexity_score, _ = DynamicComplexity()(audio16k)
    
    mel_bands = get_mel_bands(audio44k)
    novelty_curve = NoveltyCurve()(mel_bands)
    novelty_score = np.median(np.abs(np.diff(novelty_curve)))
    
    beats_loudness, beats_loudness_band_ratio = BeatsLoudness(beats=beat_positions)(audio44k)
    beatogram = Beatogram()(beats_loudness, beats_loudness_band_ratio)
    time_signature = Meter()(beatogram)

    features = {
        'Danceability': danceability_score[0],
        'Loudness': loudness_score,
        'BPM': bpm,
        'Key': key,
        'Key Scale': scale,
        'Energy': energy_score,
        'Chords Significance': chords_significance,
        'Inharmonicity': hnr_score,
        'Timbre': np.mean(mfcc_coeffs),
        'Onset Rate': onset_rate_score[1],
        'Brightness': brightness_score,
        'Dynamic Complexity': dynamic_complexity_score,
        'Novelty': novelty_score,
        'Time Signature': time_signature,
    }

    return features

In [55]:
def extract_audio_features(audio_file):
    # Load the audio file
    audio44k = MonoLoader(filename=audio_file)()
    audio16k = MonoLoader(filename=audio_file, sampleRate=16000)()

    # Run algorithms
    algorithm_features = run_essentia_algorithms(audio44k, audio16k)

    # Merge results
    return algorithm_features

#### Util Functions

In [56]:
def get_total_memory_usage(process):
    memory_summary = {f'Process {process.pid}': process.memory_info().rss / (1024 * 1024)}
    for child in process.children(recursive=True):
        memory_summary = memory_summary | {f'Child Process {child.pid}': child.memory_info().rss / (1024 * 1024)}
    return memory_summary

def print_memory_usage(process):
    print(get_total_memory_usage(process))
    snapshot = tracemalloc.take_snapshot()
    print(f"Top Consumer of Process {process.pid}: {snapshot.statistics('lineno')[0]}")

def monitor_memory_usage(process, kill_thread, interval=120):
    while True:
        try:
            if kill_thread.value:
                print("MONITOR THREAD KILLED")
                return
            print_memory_usage(process)
        except Exception as e:
            print(f"Thread ERROR: {e}")
            return
        time.sleep(interval)

#### Main Code

In [57]:
# Class constructed from song path
# Song path must follow this format: /some/path/(int)^(video id)^(title).mp3
#                               e.g  /some/path/0^LlWGt_84jpg^Special Breed.mp3
class SongPath:
    def __init__(self, song_path: str):
        self.path = song_path
        self.filename = os.path.basename(song_path)

        song_filename_split = self.filename.split('^')
        if len(song_filename_split) != 3:
            raise Exception("The song's filename doesn't follow the correct format: /some/path/(int)^(video id)^(title).mp3")
        
        self.index, self.video_id, self.title_with_extension = song_filename_split

        self.index = int(self.index)
        self.title = os.path.splitext(self.title_with_extension)[0]

    def __str__(self):
        return f"Idx: {self.index},  videoID: {self.video_id}, title: {self.title_with_extension}"

In [ ]:
def process_song(args):
    song_path, song_results = args
    song = SongPath(song_path)
    song_features = extract_audio_features(song.path)
    song_results.append((song.index, song_features))

In [ ]:
def process_songs():                   
    tracemalloc.start()

    song_paths = np.array([os.path.join(DOWNLOAD_FOLDER, song_filename) for song_filename in os.listdir(DOWNLOAD_FOLDER)])

    songs_data_lower, songs_data_higher = [len(song_paths)//6*2, len(song_paths)//6*4]
    song_paths = song_paths[songs_data_lower:songs_data_higher]
    
    with mp.Manager() as manager:
        kill_thread = manager.Value('b', False)
        shared_song_results = manager.list()

        main_process = psutil.Process(os.getpid())
        memory_thread = threading.Thread(target=monitor_memory_usage, args=(main_process, kill_thread))
        memory_thread.start()
        
        with mp.Pool(processes=CPU_THREADS, maxtasksperchild=40) as pool:
            with tqdm(total=len(song_paths), desc="Processing songs") as pbar:
                args = [(song_path, shared_song_results) for song_path in song_paths]
                try:
                    for _ in pool.imap(process_song, args, chunksize=1):
                        pbar.update(1)
                except Exception as e:
                    print(e)
                    
        kill_thread.value = True
        song_results = list(shared_song_results)

    # Aggregate results in the pandas dataframe
    songs_data_full = songs_data.copy(deep=True)
    for song_index, song_features in song_results:
        for feature, value in song_features.items():
            if feature not in songs_data_full.columns and isinstance(value, (tuple, set, list, np.ndarray, dict)):
                songs_data_full[feature] = np.nan
                songs_data_full[feature] = songs_data_full[feature].astype(object)
            songs_data_full.at[song_index, feature] = value

    return songs_data_full

In [61]:
songs_data_full = process_songs()

Main Process ID: 9865


Processing songs:   0%|          | 0/4 [00:00<?, ?it/s]

{'Process 9865': 642.03515625, 'Child Process 12225': 597.0625, 'Child Process 12236': 603.04296875, 'Child Process 12239': 599.65234375}
Top Consumer of Process 9865: /tmp/ipykernel_9865/1984634019.py:4: size=110 MiB, count=8, average=13.8 MiB
OH NO


Thread ERROR: [Errno 32] Broken pipe


In [62]:
songs_data_full.to_csv('data/songs_data_full_4.csv')

In [63]:
songs_data_full.dropna(subset=['Danceability'])


,title,artist,views,videoID,duration,Danceability,Loudness,BPM,Key,Key Scale,Energy,Chords Significance,Inharmonicity,Timbre,Onset Rate,Brightness,Dynamic Complexity,Novelty,Time Signature
0,Special Breed,PolyCulture,34,LlWGt_84jpg,331,1.365855,5662.711426,128.004791,F,minor,399511.093750,"{'A': 1368, 'Ab': 44, 'Am': 1210, 'C': 363, 'C...",0.288343,-19.498598,4.381480,1235.493164,4.430042,176.945801,4.0
100000,Gris,J Balvin,11657594,5XkHKg2Vr0M,177,1.656689,3484.364990,92.989853,F#,minor,193531.453125,"{'A': 277, 'Ab': 32, 'Abm': 57, 'Am': 13, 'B':...",0.237590,-16.540997,4.217937,1637.857544,3.100346,4340.726562,4.0
100001,La Cumbia Tribalera (feat. La Trakalosa & Viol...,El Pelon del Mikrophone,11657063,M_A58j6A-hg,218,1.347551,3015.341064,133.968338,F,minor,155969.375000,"{'A': 44, 'Ab': 141, 'Abm': 17, 'Am': 45, 'B':...",0.319716,-18.797167,5.190122,2116.453613,3.823609,4538.546875,2.0
